# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
# Importing necessary packages
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as pt

In [2]:
data = pd.io.stata.read_stata('/Users/dwreiter/Desktop/Work/Springboard/Data Stuff/EDA_racial_discrimination/data/us_job_market_discrimination.dta')

In [3]:
# Number of callbacks for black-sounding names
sum(data[data.race == 'b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1,0,1,0,0,0,0,0,0,
1,b,1,3,3,6,0,1,1,0,316,...,1,0,1,0,0,0,0,0,0,
2,b,1,4,1,6,0,0,0,0,19,...,1,0,1,0,0,0,0,0,0,
3,b,1,3,4,6,0,1,0,1,313,...,1,0,1,0,0,0,0,0,0,
4,b,1,3,3,22,0,0,0,0,313,...,1,1,0,0,0,0,0,1,0,Nonprofit


Question 1: <br>
Since this is a Bernoulli Trial where there is either success (getting called back) or failure (not getting called back), the data has a Binomial Distribution. Since we are comparing two sets of data, it is best to use a Two Sample t-test, as we do not know the standard deviation of the population. Also, the Central Limit Theorem does apply here as the data is normally distributed since resumes are randomly assigned, the sample size is greater than 30 as well as less than 10% of the total population which means we can infer independence.

Question 2: <br>
Ho = Black and White sounding resumes are receiving the same callback rate. <br>
Ha = Black and White sounding resumes are not receiving the same callback rate. <br>
Alpha = 0.05

In [5]:
w = data[data.race == 'w']
b = data[data.race == 'b']

In [6]:
# Question 3:

# Calculating the mean of callbacks for each race
m_w = np.sum(w['call']) / len(w)
m_b = np.sum(b['call']) / len(b)
print('The mean of callbacks for white-sounding resumes: ' + str(m_w))
print('The mean of callbacks for black-sounding resumes: ' + str(m_b))

# The difference between the mean of callbacks. We subtract m_b from m_w because m_w is bigger
M = m_w - m_b
print('The mean of the difference of callbacks between white and black sounding resumes: ' + str(M))

# Calculating the t-statistic and p-value
t_stat, p = stats.ttest_ind(w['call'], b['call'], equal_var=False)
print('t-statistic: ' + str(t_stat))
print('p-value: ' + str(p))

# Using the z-score for a 95% Confidence Interval, multiplying it with the computed Standard Error to get the Margin of Error
z_score = 1.96
sem = np.sqrt(np.var(w['call']) / len(w) + np.var(b['call']) / len(b))
margin_error = sem * z_score
margin_error
print('Standard Error: ' + str(sem))
print('Margin of Error: ' + str(margin_error))

# Subtract and add the Margin of Error from the difference of means for a 95% Confidence Interval
conf_int = [M - margin_error, M + margin_error]
print('95% Confidence Interval: ' + str(conf_int))

The mean of callbacks for white-sounding resumes: 0.0965092402464
The mean of callbacks for black-sounding resumes: 0.064476386037
The mean of the difference of callbacks between white and black sounding resumes: 0.0320328542094
t-statistic: 4.11470529086
p-value: 3.94294151365e-05
Standard Error: 0.00778337067153
Margin of Error: 0.0152554065162
95% Confidence Interval: [0.016777447693242267, 0.047288260725648901]


Because the p-value is lower than Alpha, we can reject Ho which means that Black and White sounding resumes are not receiving the same amount of callbacks.

Question 4: <br>
A study on the level of racial discrimination in the U.S. labor market was conducted with randomly assigned identical resumes to black-sounding or white-sounding names and observed the impact on requests for interviews from employers. Based on this study, the average amount of white-sounding resumes to get a callback was about 9.7% while black-sounding resumes only got a callback around 6.4% of the time. The difference between the amount of callbacks is around 3.2%. <br>
Although this was a sample, we can conclude with a 95% confidence interval that the true difference of means lies somewhere between around 1.7% and 4.7% with a margin of error around 1.5%. To dig deeper, we conducted a Two Sample t-test on the data and found that there was enough statistical significance to conclude that race does play a factor in applicants receiving a callback.

Question 5: <br>
While race did play a factor in applicants receiving a callback, it is not possible to claim it is the most important factor in callback success. There were many other sections on the resume that could have led to not getting a callback. In order to find if race is the most important factor, we would have to test all of the variables on the resume and find their p-values.